Questions to answer by this EDA:
- What kind of data do we have? 
    - What kind of data do we have. Numeric, images, time series?  
    - How big is the dataset?
- What can l5kit do?
    - Can we combine zarr files?
    - other functionality that might prove useful
- General visualization and analysis
    - How is the data distributed?
    - Is the data complete?
    - Does the data need any cleaning?

In [11]:
import numpy
import pandas
import l5kit

from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset

from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points
from tqdm import tqdm
from collections import Counter
from l5kit.data import PERCEPTION_LABELS
from prettytable import PrettyTable

In [12]:
os.environ["L5KIT_DATA_FOLDER"] = "/home/elias/Documents/lyft-motion-prediction/data"

cfg = load_config_data("./eda_config.yaml")

print(f'current raster_param:\n')
for k,v in cfg["raster_params"].items():
    print(f"{k}:{v}")


current raster_param:

raster_size:[224, 224]
pixel_size:[0.5, 0.5]
ego_center:[0.25, 0.5]
map_type:py_semantic
satellite_map_key:aerial_map/aerial_map.png
semantic_map_key:semantic_map/semantic_map.pb
dataset_meta_key:meta.json
filter_agents_threshold:0.5
disable_traffic_light_faces:False


# Loading the data

In [16]:
dm = LocalDataManager()
sample_dataset_path = dm.require(cfg["sample_data_loader"]["key"])
train_dataset_path = dm.require(cfg["train_data_loader"]["key"])
validate_dataset_path = dm.require(cfg["val_data_loader"]["key"])

sample_dataset = ChunkedDataset(sample_dataset_path)
train_dataset = ChunkedDataset(train_dataset_path)
validate_dataset = ChunkedDataset(validate_dataset_path)

print("SAMPLE DATASET")
sample_dataset.open()
print(sample_dataset)

print("TRAINING DATASET")
train_dataset.open()
print(train_dataset)

print("VALIDATION DATASET")
validate_dataset.open()
print(validate_dataset)

SAMPLE DATASET
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|    100     |   24838    |  1893736   |     316008    |       0.69      |        248.38        |        76.24         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
TRAINING DATASET
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+------------------

The validation dataset seems to be just as large as the training dataset. For training of the final model, this data should definately be included in training

&lt;l5kit.data.zarr_dataset.ChunkedDataset at 0x7fbccdd57ac0&gt;